In [ ]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_community.chat_models import ChatOllama

In [ ]:
# Set environment variables for Google Cloud and OpenAI
service_account_file  = "./credentials/google.json"
os.environ["OPENAI_API_KEY"] = "API_KEY"

project = 'rf-oss'
dataset = "dailydata"

In [ ]:
# SQLAlchemy connection
sqlalchemy_url = f'bigquery://{project}/{dataset}'
print(sqlalchemy_url)

# Create an Engine and SQLDatabaseToolkit 
db = SQLDatabase.from_uri(sqlalchemy_url , {"credentials_path": service_account_file})
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    api_key='API_KEY'
)
db.run()
# llm = ChatOpenAI(
#     model="gpt-4o", 
#     api_key='API_KEY'
# )
# llm = ChatOllama(model="llama3" , temperature=0)
# toolkit = SQLDatabaseToolkit(db=db , llm=llm)
# agent_exec = create_sql_agent(
#     llm=llm , 
#     toolkit=toolkit, 
#     verbose=True, 
#     top_k=10
# )

In [ ]:
examples = [
    {
        "input": "How is the performance 4G Volte Drop Call rate for site 1123A for past 1 weeks", 
        "query": "SELECT time , 100*sum(VOLTE_DROP_NUM)/sum(VOLTE_DROP_DENOM) `4G VoLTE Drop Call Rate` FROM `rf-oss.dailydata.mc4g_cell_combined` WHERE Date(time) >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND ( site = '1123A' ) GROUP BY time ORDER BY time"
    }, 
    {
        "input": "How is the performance 2G TCH Drop rate for site 1123A for past 1 month", 
        "query": "SELECT time , 100*sum(TCH_DROP_NUM)/sum(TCH_DROP_DENOM) `2G TCH Drop Rate` FROM `rf-oss.dailydata.mc2g_cell_combined` WHERE Date(time) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AND ( site = '1123A' ) GROUP BY time ORDER BY time"
    }, 
    {
        "input": "How much total 4G data download traffic for the site 1011A for past 2 week", 
        "query": "SELECT sum(LTE_DATA_TRAFFIC_GB) `4G Data Download Traffic` FROM `rf-oss.dailydata.mc4g_cell_combined` WHERE Date(time) >= DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY) AND ( site = '1011A' )"
    }, 
    {
        "input": "Which cell is having the worst performance 4G RRC setup success rate for 1011A in last 3 days", 
        "query": "SELECT object `Cell` , 100*sum(RRC_SETUP_NUM)/sum(RRC_SETUP_DENOM) `RRC Setup Success Rate` FROM `rf-oss.dailydata.mc4g_cell_combined` WHERE Date(time) >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 DAY) AND ( site = '1011A' ) GROUP BY object ORDER BY `RRC Setup Success Rate` LIMIT 1;"
    }
]

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate , PromptTemplate

prompt_template = """ 
You are a SQLite expert with Telecommunication RF Domain Expert. Given an input question, create a syntactically correct SQLite query to run. Generate the SQL in single line. \
You must know which technology context user is asking, either 4G (LTE) or 2G (GSM). If user didn't provide the context, don't run generate SQL and run SQL.
Unless otherwise specificed, do not return more than {top_k} rows.\n\n \
Here is the relevant table info: {table_info}\n\n \

Below are a number of examples of questions and their corresponding SQL queries.
"""
example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples[:5], 
    example_prompt=example_prompt, 
    prefix=prompt_template , 
    suffix="User input: {input}\nSQL query: ", 
    input_variables=["input" , "top_k" , "table_info"]
)

In [ ]:
print(prompt.format(input="How is the performance 4G CSSR for past 7 days for site 1011A?" , top_k=3 , table_info="mc4g_cell_combined"))

In [ ]:
from langchain.globals import set_debug , set_verbose
set_verbose(False)
set_debug(False)
chain = create_sql_query_chain(llm , db , prompt)
# chain.invoke({"question": "How is the 4G CSSR performance for 1011A for last 14 days?", "table_names_to_use":['mc4g_cell_combined']})
chain.invoke({"question": "How is the 4G Volte CSSR performance for 1011A for last 14 days?", "table_names_to_use": ['mc4g_cell_combined' , 'mc2g_cell_combined']})

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool 
from langchain.output_parsers import PandasDataFrameOutputParser 
from langchain.output_parsers.json import SimpleJsonOutputParser

def parse(message):
    """Parse output in pretty format"""
    message = message.split("), (")
    return message
parser = SimpleJsonOutputParser()
execute_query = QuerySQLDataBaseTool(db=db , return_direct=True)
full_chain = chain | execute_query | parse

In [ ]:
full_chain.invoke({"question": "How is the 4G CSSR performance for 1011A for last 14 days?", "table_names_to_use":['mc4g_cell_combined']} )

In [ ]:
full_chain.invoke({"question": "How is the 2G CSSR performance for 1011A for last 3 days?" , "table_names_to_use":["mc4g_cell_combined" , 'mc2g_cell_combined']})

In [ ]:
full_chain.invoke({"question": "List me top 3 cell that have the poor 2G CSSR for 1011A for last 3 days" , "table_names_to_use":["mc4g_cell_combined"]})

In [ ]:
full_chain.invoke({"question": "What is the total voice traffic for 1101A for last 8 days" , "table_names_to_use":["mc4g_cell_combined"]})

In [ ]:
full_chain.invoke({"question": "How much is the total call drop for 1101A for past 3 days and which cell is giving the poorest result" , "table_names_to_use": ["mc4g_cell_combined" , "mc2g_cell_combined"]})

In [ ]:
full_chain.invoke({"question": "Get me the total data traffic for 1101A for last 5 days. " , "table_names_to_use":["mc4g_cell_combined"]})

In [ ]:
full_chain.invoke({"question": "How is the volte drop call rate for 1011A for last 14 days?" , "table_names_to_use":["mc4g_cell_combined"]})

In [1]:
from telco_llm.utils.common import create_telco_sql_query_chain

In [2]:
chain = create_telco_sql_query_chain()

NoInspectionAvailable: No inspection system is available for object of type <class 'str'>